In [18]:
! pip install datasets
! pip install transformers
! pip install evaluate
! pip install numpy
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.6 MB/s eta 0:00:

In [19]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [20]:
from datasets import load_dataset, DatasetDict

train_ds, test_ds = load_dataset('JBJoyce/DENTAL_CLICK', split=['train', 'test'])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3956 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1020 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/JBJoyce___parquet/JBJoyce--DENTAL_CLICK-1463b76376495121/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
trainvalid_ds = train_ds.train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': trainvalid_ds['train'],
    'test': test_ds,
    'valid': trainvalid_ds['test']})

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 3164
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 1020
    })
    valid: Dataset({
        features: ['audio', 'label'],
        num_rows: 792
    })
})

In [23]:
label2id = {'OTHER': 0, 'DENTAL_CLICK': 1}
id2label = {v:k for k,v in label2id.items()}

In [24]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("JBJoyce/wav2vec2-base-superb-ks")

In [25]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [26]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [27]:
encoded_dataset = dataset.map(preprocess_function, remove_columns='audio', batched=True)

Map:   0%|          | 0/3164 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Map:   0%|          | 0/792 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [28]:
import evaluate

accuracy = evaluate.load("accuracy")

In [29]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [30]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "JBJoyce/wav2vec2-base-superb-ks", 
    num_labels=num_labels, 
    label2id=label2id, 
    id2label=id2label,
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at JBJoyce/wav2vec2-base-superb-ks and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([31, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([31]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from torch import device, cuda
device = device("cuda") if cuda.is_available() else device("cpu")
model.to(device)
device

device(type='cuda')

In [35]:
training_args = TrainingArguments(
    output_dir="output_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['test'],
    eval_dataset=encoded_dataset['valid'],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [36]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.215172,0.958333
2,0.439000,0.181654,0.958333
3,0.136000,0.178464,0.958333
4,0.139600,0.169410,0.958333
5,0.135400,0.161112,0.958333
6,0.135400,0.135763,0.958333
7,0.118500,0.125035,0.960859
8,0.085200,0.102439,0.968434
9,0.076800,0.103411,0.965909
10,0.067200,0.109802,0.958333


TrainOutput(global_step=80, training_loss=0.14971186965703964, metrics={'train_runtime': 183.5648, 'train_samples_per_second': 55.566, 'train_steps_per_second': 0.436, 'total_flos': 9.2602052928e+16, 'train_loss': 0.14971186965703964, 'epoch': 10.0})

In [37]:
trainer.predict(encoded_dataset['test'])

PredictionOutput(predictions=array([[ 0.01402012, -0.23519407],
       [ 1.7998257 , -2.166097  ],
       [ 2.1728969 , -2.4534593 ],
       ...,
       [ 2.1830077 , -2.4556527 ],
       [ 2.1595783 , -2.4267726 ],
       [ 2.203099  , -2.4480608 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.0574534572660923, 'test_accuracy': 0.9823529411764705, 'test_runtime': 8.0295, 'test_samples_per_second': 127.032, 'test_steps_per_second': 3.985})

In [39]:
from transformers import pipeline
trained_model = "output_dir/checkpoint-80"
pipe = pipeline("audio-classification", 
                model=trained_model, 
                feature_extractor=feature_extractor,
                device="cpu")

In [58]:
from IPython.display import Audio
first_click_audio = dataset['test'].filter(lambda example: example['label'] == 1)[0]['audio']['array']
Audio(first_click_audio, rate=16000)

In [59]:
pipe(first_click_audio)

[{'score': 0.7217940092086792, 'label': 'DENTAL_CLICK'},
 {'score': 0.2782059609889984, 'label': 'OTHER'}]

In [67]:
first_non_click_audio = dataset['test'].filter(lambda example: example['label'] == 0)[0]['audio']['array']
Audio(first_non_click_audio, rate=16000)

In [68]:
pipe(first_non_click_audio)

[{'score': 0.603608250617981, 'label': 'DENTAL_CLICK'},
 {'score': 0.39639177918434143, 'label': 'OTHER'}]

In [43]:

Audio(dataset['test'][1]['audio']['array'], rate=16000)

In [69]:
from huggingface_hub import Repository

repo = Repository("for_clone", clone_from="JBJoyce/DENTAL_CLICK_classifier")

Cloning https://huggingface.co/JBJoyce/DENTAL_CLICK_classifier into local empty directory.


In [70]:
model_for_upload = AutoModelForAudioClassification.from_pretrained(trained_model)
feat_ext_for_upload = AutoFeatureExtractor.from_pretrained(trained_model)

In [71]:
repo.git_pull()

In [72]:
model_for_upload.save_pretrained("for_clone")
feat_ext_for_upload.save_pretrained("for_clone")

['for_clone/preprocessor_config.json']

In [73]:
repo.git_add()
repo.git_commit("Add model and feature extractor")
repo.git_push()

Upload file pytorch_model.bin:   0%|          | 32.0k/361M [00:00<?, ?B/s]

To https://huggingface.co/JBJoyce/DENTAL_CLICK_classifier
   918e003..8bc7ff3  main -> main

   918e003..8bc7ff3  main -> main



'https://huggingface.co/JBJoyce/DENTAL_CLICK_classifier/commit/8bc7ff36b5b7756ddee83b766edb303f68600769'